In [1]:
#!pip freeze | grep scikit-learn

In [2]:
#!python -V

In [2]:
#!wget https://github.com/DataTalksClub/mlops-zoomcamp/raw/refs/heads/main/cohorts/2025/04-deployment/homework/model.bin

In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
#!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet

In [7]:
year=2023
month=3

df = read_data('yellow_tripdata_2023-03.parquet')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [9]:
y_pred.std()

np.float64(6.247488852238703)

In [11]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df['duration_prediction']=y_pred

In [12]:
df_result=df[ ['ride_id','duration_prediction'] ]

In [13]:
df_result

,ride_id,duration_prediction
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486
...,...,...
3403761,2023/03_3403761,11.952635
3403762,2023/03_3403762,20.049958
3403763,2023/03_3403763,11.595336
3403764,2023/03_3403764,13.113178


In [15]:
df_result.to_parquet(
    'df_result.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

In [16]:
!ls  -alh

total 120M
drwxr-xr-x 3 mahdi mahdi 4.0K Jun 14 19:40 .
drwxr-xr-x 3 mahdi mahdi 4.0K Jun 14 19:30 ..
drwxr-xr-x 2 mahdi mahdi 4.0K Jun 14 19:31 .ipynb_checkpoints
-rw-r--r-- 1 mahdi mahdi  66M Jun 14 19:40 df_result.parquet
-rw-r--r-- 1 mahdi mahdi  17K Jun 14 19:31 model.bin
-rw-r--r-- 1 mahdi mahdi 7.8K Jun 14 19:39 starter.ipynb
-rw-r--r-- 1 mahdi mahdi  54M May 19  2023 yellow_tripdata_2023-03.parquet


In [17]:
# !jupyter nbconvert --to script starter.ipynb

In [ ]:
# !pipenv install scikit-learn==1.5.0 pandas pyarrow tqdm  fire requests

In [3]:
# hash: 057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c

In [ ]:
# predicted duration mean: 14.292282936862449

In [ ]:
# !docker build  -t mlops_04_batch_script .

In [ ]:
# !docker run -it mlops_04_batch_script --year=2023 --month=5

In [ ]:
# predicted duration mean: 0.19174419265916945